# Project 5

In the following exercise, we will perform bootstrap and cross validation to find statistical values for two datasets.

In this assignment, we will be using PennGrader, a Python package built by a former TA for autograding Python notebooks. PennGrader was developed to provide students with instant feedback on their answer. You can submit your answer and know whether it's right or wrong instantly. We then record your most recent answer in our backend database. You will have 100 attempts per test case, which should be more than sufficient.

<b>NOTE：Please remember to remove the </b>

```python
raise notImplementedError
```
<b>after your implementation, otherwise the cell will not compile.</b>


A few things to note before we start
- You do not have to round your answers.
- Please make sure your answer's variable name do match what is provided in grader cells.

## Getting Setup
Please run the below cells to get setup with the autograder. If you need to install packages, please do it below!

In [2]:
# %%capture
# !pip install penngrader --user

In [3]:
# !pip install scikit-learn --user
# !pip install statsmodels

Let's try PennGrader out! Fill in the cell below with your PennID and then run the following cell to initialize the grader.

<font color='red'>Warning:</font> Please make sure you only have one copy of the student notebook in your directory in Codio upon submission. The autograder looks for the variable `STUDENT_ID` across all notebooks, so if there is a duplicate notebook, it will fail.

In [4]:
#PLEASE ENSURE YOUR STUDENT_ID IS ENTERED AS AN INT (NOT A STRING). IF NOT, THE AUTOGRADER WON'T KNOW WHO 
#TO ASSIGN POINTS TO YOU IN OUR BACKEND

STUDENT_ID = 56803282                   # YOUR 8-DIGIT PENNID GOES HERE
STUDENT_NAME = "Jacky Choi"     # YOUR FULL NAME GOES HERE

In [5]:
import penngrader.grader

grader = penngrader.grader.PennGrader(homework_id = 'ESE542_Online_Su_2021_HW5', student_id = STUDENT_ID)

## Part A

First, we will use the Boston dataset to try running our own implementation of bootstrap. 

In [6]:
#Data Wrangling
import pandas as pd
import numpy as np
import math
#Logistic Regression
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression

#Plotting
import matplotlib.pyplot as plt

#Cross Validation
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

#Statistics
from scipy import stats

RANDOM_STATE=42
%matplotlib inline

In [7]:
from sklearn.datasets import load_boston
boston_dataset = load_boston()
data = pd.DataFrame(data = boston_dataset.data, columns = boston_dataset.feature_names)
data['MEDV'] = pd.Series(boston_dataset.target)
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Based on this dataset, provide an estimate for $\hat\mu$, the population mean of 'medv'. Name this variable `mu_hat`.

In [8]:
# Enter your code below

mu_hat = data["MEDV"].mean()
print(mu_hat)

22.532806324110677


In [9]:
grader.grade(test_case_id = 'test_mu_hat', answer = mu_hat)

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


2. Provide an estimate of the standard error of $\hat\mu$. Name this variable `se_mu_hat`>. Interpret this result in your Jupyter notebook. 

*Hint*: We can compute the standard error of the sample mean by dividing the <u>sample</u> standard deviation by the square root of the number of observations. When computing the sample standard deviation, one must consider the Bessel’s correction factor from statistics (more information on that [here](https://en.wikipedia.org/wiki/Bessel\%27s_correction))

In [10]:
# Enter your code below

# se_mu_hat = data["MEDV"].sem() also works
se_mu_hat = data["MEDV"].std() / math.sqrt(len(data['MEDV']))
print(se_mu_hat)

0.40886114749753505


In [11]:
grader.grade(test_case_id = 'test_se_mu_hat', answer = se_mu_hat)

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


3. Now estimate the standard error of  by implementing your own bootstrap method. Name your final answer `se_boostrap`. How does this compare to your answer from Part A Question 2? 

*Hint*: Generate $M=n$ artificial datasets, each with $N=n$ bootstrap samples. Use `np.random.seed(RANDOM_STATE)` before sampling so that your outputs stay consistent. You should try to write your own bootstrap method and and then compare it to packages such as SKLearn's `resample`.

In [12]:
# Enter your code below
np.random.seed(42)
medv = data["MEDV"]
bootstrap_means = []
for i in range(len(medv)):
    bs_sample = np.random.choice(medv, size = len(medv), replace=True)
    bs_mean = np.mean(bs_sample)
    bootstrap_means.append(bs_mean)
    
se_bootstrap = np.std(bootstrap_means, ddof=1)
print(se_bootstrap)

0.4065800260905055


In [14]:
grader.grade(test_case_id = 'test_se_bootstrap', answer = se_bootstrap)

Correct! You earned 2/2 points. You are a star!

Your submission has been successfully recorded in the gradebook.


4. Based on your bootstrap estimate from Part A Question 3, provide a $95\%$ confidence interval for the mean of 'medv'. Enter the lower range, higher range for mean's confidence interval in CI_lower, CI_upper.

*Hint*: You can approximate a $95\%$ confidence interval using the formula $[\hat\mu \pm 2 * SE(\hat\mu)] $.

In [15]:
# Enter your code below

CI_lower = mu_hat - 2 * se_bootstrap
CI_upper = mu_hat + 2 * se_bootstrap
print(CI_lower, CI_upper)

21.719646271929665 23.34596637629169


In [16]:
grader.grade(test_case_id = 'test_CI_mean', answer = (CI_lower, CI_upper))

Correct! You earned 1.0/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


5. Based on this data set, provide the population median value of 'medv', $\widehat{median}$. Name this variable `median_hat`.

In [17]:
# Enter your code below

median_hat = data['MEDV'].median()
print(median_hat)

21.2


In [18]:
grader.grade(test_case_id = 'test_median_hat', answer = median_hat)

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


6. We now would like to estimate the standard error of $\widehat{median}$. Unfortunately, there is no simple formula for computing the standard error of the median. Instead, estimate the standard error of the median using bootstrap. Name this variable `se_bootstrap_median`.

*Hint*: use your bootstrap sample list from A.3.

In [19]:
# Enter your code below
np.random.seed(42)
medv = data["MEDV"]
bootstrap_medians = []
for i in range(len(medv)):
    bs_sample = np.random.choice(medv, size = len(medv), replace=True)
    bs_median = np.median(bs_sample)
    bootstrap_medians.append(bs_median)
    
se_bootstrap_median = np.std(bootstrap_medians, ddof=1)
print(se_bootstrap_median)

0.35947404421347856


In [20]:
grader.grade(test_case_id = 'test_se_median', answer = se_bootstrap_median)

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


7.  Is the standard error of the median greater or less than the standard error of the mean?Does this make intuitive sense? Comment on your other findings in the cell below

In [21]:
# Add comment here
#The standard error for the median about 0.5 less than the standard error for the mean. 
#This could be affected by outliers or different kinds of distributions espeically for heavily skewed data distributions.
#Usually a lower SE for median compared to mean shows there are meany outliers.

## Part B

Next, we will use the Default dataset to predict the probability of default using income and balance. In doing so, we also want to estimate the test error of the logistic regression model described in that section using cross validation.

In [49]:
default = pd.read_csv('Default.csv')
default.head(5)


,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879


1. Fit a logistic regression model that use <u>all</u> the data from the predictors 'income' and 'balance' to predict 'default'. Please name your model <b>fit1</b>.

*Hint*: Use `Stats Models` to fit the logistic regression. Review the previous recitations for review. 

In the latest versions of Stats Models, the default encoding for 'No' and 'Yes' were changed. Before running logistic regression, please make sure that the 'default' and 'student' columns are encoded so that 'No' = 0 and 'Yes' = 1. 

In [50]:
# Enter your code below

default['default'] = default['default'].apply(lambda x: 1 if x =='Yes' else 0)
default['student'] = default['student'].apply(lambda x: 1 if x =='Yes' else 0)
fit1 = smf.glm('default~balance+income', data=default, family=sm.families.Binomial()).fit()
# fit1 = smf.ols('default~balance+income', data = default).fit()
fit1.summary()



<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                default   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9997
Model Family:                Binomial   Df Model:                            2
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -789.48
Date:                Thu, 10 Oct 2024   Deviance:                       1579.0
Time:                        23:40:44   Pearson chi2:                 6.95e+03
No. Iterations:                     9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -11.5405      0.435    -26.544      0.000     -12.393     -10.688
balance        0.0056      0.000     24.835      0.000       0.005       0.006
income      2.081e-05   4.99e-06      4.174      0.000     1.1e-05    3.06e-05
==============================================================================
"""

In [51]:
grader.grade(test_case_id ='test_glm_type', answer = str(type(fit1)))

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


In [52]:
grader.grade(test_case_id ='test_glm', answer = fit1.params)

Correct! You earned 2/2 points. You are a star!

Your submission has been successfully recorded in the gradebook.


2. Using the validation set approach, estimate the test error of this model. In order to do this, you must perform the following steps: 


- Split the sample set into a random training set and a random validation set. Use a test size of 20%. *Hint*: Use `sklearn.model_selection.train_test_split` and a `random_state` of 42/RANDOM_STATE. Store your training set in <b>train</b>, testing set in <b>test</b>.

In [53]:
# Enter your code below

train, test = train_test_split(default, test_size=0.2, random_state=42)


In [54]:
grader.grade(test_case_id ='test_train_shape', answer = train)

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


In [55]:
grader.grade(test_case_id ='test_test_shape', answer = test)

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


- Fit a logistic regression with multiple variables model using <u>only</u> the training observations. Name your model <b>fit2</b>.

In [83]:
# Enter your code below

fit2 = smf.glm('default~income+balance', data=train, family=sm.families.Binomial()).fit()
fit2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                default   No. Observations:                 5000
Model:                            GLM   Df Residuals:                     4997
Model Family:                Binomial   Df Model:                            2
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -392.46
Date:                Thu, 10 Oct 2024   Deviance:                       784.93
Time:                        23:56:20   Pearson chi2:                 4.15e+03
No. Iterations:                     9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -11.9681      0.640    -18.688      0.000     -13.223     -10.713
income      1.934e-05   6.99e-06      2.766      0.006    5.63e-06     3.3e-05
balance        0.0060      0.000     17.665      0.000       0.005       0.007
==============================================================================
"""

In [57]:
grader.grade(test_case_id ='test_glm_type', answer = str(type(fit2)))

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


In [58]:
grader.grade(test_case_id ='test_train_para', answer = fit2.params)

Correct! You earned 1.0/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


- Obtain a prediction of 'default' for each individual in the test set by computing the probability of default for that individual and classifying the individual to the default category if the posterior probability is greater than or equal to 0.5 (a Bayesian classifier). Store your predictions in <b>predicted</b>.

*Hint*: we check for both the size and values of your precitions!

In [59]:
# Enter your code below

pred = fit2.predict(test)
predicted = pred.apply(lambda x: 1 if x >= 0.5 else 0)
predicted

6252    0
4684    0
1731    0
4742    0
4521    0
       ..
6412    0
8285    0
7853    0
1095    0
6929    0
Length: 2000, dtype: int64

In [60]:
grader.grade(test_case_id ='test_glm_predict', answer = predicted)

Correct! You earned 1.0/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


- Compute the misclassification rate, which is the fraction of the observations in the validation set that are misclassified. Name this variable `mis_rate` and. 

*Hint*: Compare the predictions with the test set. Your answer should be the ratio of number of incorrect predictions over the total number of samples.

In [61]:
# Enter your code below
incorrect = (predicted != test['default']).sum()
mis_rate = (incorrect / len(test))
mis_rate

0.0305

In [62]:
grader.grade(test_case_id ='test_mis_rate', answer = mis_rate)

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


3. Repeat the process in Part B Question 2 three times, using three different splits of the observations into a training set and a validation set. Use test sizes of $10\%$, $30\%$, and $50\%$. Name the misclassification rates `mis_rate_10`, `mis_rate_30`, `mis_rate50`, respectively. Comment on the results obtained and determine the key takeaway. Store the misclassification accuracy score for each model within <b>model_rates</b> and model parameters within <b>model_params</b>

*Note*: When splitting to test/train sets, please set your random_state to RANDOM_STATE so that you can have the same group of testing and training sets.

In [63]:
# Enter your code below
RANDOM_STATE=42
test_sizes = [0.1, 0.3, 0.5]
model_rates = []
for i in test_sizes:
    train, test = train_test_split(default, test_size=i, random_state=RANDOM_STATE)
    fit3 = smf.glm('default~income+balance', data=train, family=sm.families.Binomial()).fit()
    pred = fit3.predict(test)
    predicted = pred.apply(lambda x: 1 if x >= 0.5 else 0)
    incorrect = (predicted != test['default']).sum()
    mis_rate = (incorrect / len(test))
    print(mis_rate)

    model_rates.append(mis_rate)



0.033
0.02666666666666667
0.0258


In [64]:
grader.grade(test_case_id ='test_models_rates', answer = model_rates)

Correct! You earned 2.0/2 points. You are a star!

Your submission has been successfully recorded in the gradebook.


4. Using KFold cross validation with five folds across 5 trials, calculate the <b>average</b> misclassification rate. Name this `mis_rate_kfold`. In your notebook, briefly explain the KFold process. 

*Hint*: Because `Stats Models` does not have its own cross validation libraries, you may have to use the following packages (specify a `random_state` equaling the trial number on each trial (0,1,2,3,4)): 


- `sklearn.linear_model.LogisticRegression`
- `sklearn.model_selection.cross_val_score`
- `sklearn.model_selection.KFold`

In [80]:
# Enter your code below
X = default[['income', 'balance']]
y = default['default']
miss_rates = []
model = LogisticRegression() 


for trial in range(5):
    cv_method = KFold(n_splits=10, shuffle=True, random_state=trial)
    scores = cross_val_score(model, X, y, cv=cv_method, scoring = 'accuracy')
    miss = 1-scores
    avg_miss = miss.mean()
    miss_rates.append(avg_miss)

mis_rate_kfold = sum(miss_rates) / len(miss_rates)
mis_rate_kfold



0.029980000000000024

In [81]:
grader.grade(test_case_id ='test_mis_kfold', answer = mis_rate_kfold)

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


5. Using bootstrap on the complete dataset, compute estimates for the standard errors of the `income` and `balance` logistic regression coefficients. In order to do this, you should perform the following steps:


- Write a function that takes as input the Default data set (and optionally, an index of the observations). The method should be called `boot_sample(data,index)`. The method should return the coefficient estimates for income and balance in the logistic regression model.

In [82]:
def boot_sample(data, index=None):
    """ Use bootstrap to compute estimates for the standard
        errors of the income and balance logistic regression
        
        data: training data
        index: sample indexes from bootstrap, optional
    """
    
   # Enter your code below
    model = smf.glm(formula='default ~ income + balance', data=data, family=sm.families.Binomial()).fit()
    coef_income = model.params[1]
    coef_balance = model.params[2]
    return [coef_income, coef_balance]

b. Use this function to estimate the standard errors of the logistic regression coefficients for income and balance using 100 different bootstrap samples. Store your lists of results in boot_income, boot_balance, both of lists should be of size (100,1). 
*Note*: Do not set a random seed in this problem, otherwise you will not be randomly sample from the data.

In [68]:
length = 100
boot_income = []
boot_balance = []
def Avg(l): 
    return sum(l) / len(l) 
# Enter your code below
for i in range(length):
    coef = boot_sample(default, len(default))
    boot_income.append(coef[0])
    boot_balance.append(coef[1])
boot_income_avg = Avg(boot_income)
boot_balance_avg = Avg(boot_balance)
print(boot_income_avg, boot_balance_avg)

2.0808975528986823e-05 0.0056471029503164915


In [69]:
grader.grade(test_case_id ='test_boot_income_balance', answer = (boot_income, boot_balance))

Correct! You earned 2/2 points. You are a star!

Your submission has been successfully recorded in the gradebook.


- Comment on the estimated standard errors obtained. What are the confidence intervals of the logistic regression coefficients? 

In [79]:
# Enter your comment here
#The standard errors are very small.
ci_income_lower = boot_income_avg - 2*boot_income_avg
ci_income_upper = boot_income_avg + 2*boot_income_avg
print(ci_income_lower, ci_income_upper)
ci_balance_lower = boot_balance_avg - 2*boot_balance_avg
ci_balance_upper = boot_balance_avg + 2*boot_balance_avg
print(ci_balance_lower, ci_balance_upper)

-2.0808975528986823e-05 6.242692658696047e-05
-0.0056471029503164915 0.016941308850949473


Congratulations on making the end of project 5. Please remember to save a copy of .ipynb and .html of your notebook, and mark your project as complete on Codio.